# NLP HSE School

In [233]:
from pymystem3 import Mystem
import pandas as pd
import re
m=Mystem()
pd.set_option('display.width', 1000)
pd.set_option('display.expand_frame_repr', False)

In [7]:
sentilex = pd.read_csv('RuSentiLex2017_revised_2utf.txt',
                       names=['word','part','initial','sentiment','source','amb1','amb2'],
                       skiprows=20,index_col=2,quotechar='"',quoting=2,skipinitialspace=True)
sentilex['sentiment'] = sentilex['sentiment'].map({'negative':-1,'positive':1,'neutral':0,'positive/negative':0})

In [222]:
vocab = sentilex.to_dict(orient='index')


In [223]:

def read_file(n):
    file = pd.read_csv('Texts/art{}.txt'.format(n),sep='\n',names=['lines'])
    file.lines = file.lines.str.replace("{Author, Unknown} ","")
    
    ent_df = pd.read_csv('Texts/art{}.ann'.format(n),sep='\t',names=['ind','descr','ne'],header=None)
    ent_df = ent_df.set_index(ent_df['ne'].apply(lambda x: m.lemmatize(x.lower())[0]))
    syn_df = pd.read_json('Texts/art{}syn.json'.format(n))

    syn_df = syn_df.reset_index()
    def get_struct_data(x):
        sentl =[]
        pars=[]
        for text in x:
            for sent in text:
                sentl = sent.split(' ')
                to_app = [sentl[i] for i in range(len(sentl)) if i in([0,1,4,6,8,10,12])]
                to_app.append(m.lemmatize(to_app[2])[0])
                pars.append(to_app)
        return(pars)

    syn_df['syntax'] = syn_df['syntax'].apply(get_struct_data)
    
    syn_df['index']=syn_df['index'] +1
    file = file.reset_index().merge(syn_df)
    return(file.drop('text',1), ent_df.to_dict(orient='index'))


In [224]:
import re
def sentiment_extraction(x,ent_dict,syn_df):
    line = x['lines']
    syntax = x['syntax']
    ne_list = []
    for ne in ent_dict.keys():
        regex= re.compile(r"\b{}\b".format(re.escape(ent_dict[ne]['ne'])),flags=re.UNICODE)
        for pos in ([m.start(0) for m in re.finditer(regex, line)]):
            ne_list.append((ne,pos))
    ne_pairs  = [(ne1,ne2) for ne1 in ne_list for ne2 in ne_list if ne1 != ne2]
    
    for ((ne1, ne1_pos), (ne2,ne2_pos)) in ne_pairs:
        x['mutual'] = get_mutual_node(syntax,ent_dict[ne1]['ne'], ne1_pos, ent_dict[ne1]['ne'], ne2_pos)
                   
    x['ne_pairs'] = ne_pairs
    
    return(x)

In [225]:
#получение данных из синтаксиса
def get_mutual_node(syntax,ent1,ent1pos,ent2,ent2pos):
    
    def get_node(e,p):
        
        e = e.split(" ")[0].split("-")[0]
        ent_dict = dict()
        for i in range(len(syntax)):
            if(syntax[i][2]==e):
                
                ent_dict[abs(int(syntax[i][1])-p)]=i
        
        return ent_dict[sorted(ent_dict)[0]]
    def get_route(i):
        route = list()
        for _ in range(len(syntax)):
            h=int(syntax[i][5])
            if(h ==-1): return route
            route.append(syntax[h])
            i=h
    ent_node1 = get_node(ent1,ent1pos)
    ent_node2 = get_node(ent2,ent2pos)
    
    route1= get_route(ent_node1)[::-1]
    route2= get_route(ent_node2)[::-1]
    if(len(route1)==0): return syntax[ent_node1],syntax[ent_node1],syntax[ent_node2]
    if(len(route2)==0): return syntax[ent_node2],syntax[ent_node1],syntax[ent_node2]

    for i in range(min(len(route1),len(route2))):
        
        if(route1[i][0]==route2[i][0]): obj=route1[i]
        else: break
    
    return obj,syntax[ent_node1],syntax[ent_node2]

In [235]:
text_lines, ent_dict = read_file(6)

text_lines.apply(lambda x:sentiment_extraction(x,ent_dict,syn_df),axis=1)


,index,lines,mutual,ne_pairs,syntax
0,1,"inosmi.ruПосмотреть оригиналапрель 29-го, 2016",NaN,[],"[[0, 6, inosmi, ADJ, Animacy=Inan|Case=Nom|Gen..."
1,2,По оценкам главы комиссии парламента Эстонии п...,NaN,[],"[[0, 2, По, ADP, fPOS=ADP++, 1, case, по], [3,..."
2,3,"Путин хочет изменить мировой порядок, во главе...","([94, 100, Андрес, NOUN, Animacy=Anim|Case=Nom...","[((путин, 0), (сша, 62)), ((путин, 0), (нато, ...","[[0, 5, Путин, NOUN, Animacy=Anim|Case=Nom|Gen..."
3,4,Глава комиссии по иностранным делам эстонского...,"([100, 104, Свен, NOUN, Animacy=Anim|Case=Nom|...","[((путин, 169), (россия, 203)), ((путин, 169),...","[[0, 5, Глава, NOUN, Animacy=Inan|Case=Nom|Gen..."
4,5,"«Он не просыпается каждое утро с мыслью о том,...",NaN,[],"[[0, 1, «, PUNCT, fPOS=PUNCT++,, 3, dobj, «\n]..."
5,6,"С другой стороны, он принимает решения, не пре...","([115, 125, посольстве, NOUN, Animacy=Inan|Cas...","[((миксер, 75), (хельсинки, 136)), ((хельсинки...","[[0, 1, С, ADP, fPOS=ADP++, 2, case, с], [2, 8..."
6,7,Российская внешняя политика временами непредск...,"([62, 68, Андрес, NOUN, Animacy=Anim|Case=Nom|...","[((андрес, 62), (служба, 99)), ((андрес, 62), ...","[[0, 10, Российская, ADJ, Case=Nom|Degree=Pos|..."
7,8,"Он напоминает, что захват Крыма и начало бомба...","([41, 54, бомбардировок, NOUN, Animacy=Inan|Ca...","[((крым, 26), (сирия, 57)), ((сирия, 57), (кры...","[[0, 2, Он, PRON, fPOS=PRON++, 1, nsubj, он], ..."
8,9,"«В глазах России балтийские страны, конечно, о...",NaN,[],"[[0, 1, «, PUNCT, fPOS=PUNCT++,, 1, punct, «\n..."
9,10,"Россия может проверить это», — предупреждает В...",NaN,[],"[[0, 6, Россия, NOUN, Animacy=Inan|Case=Nom|Ge..."


Перенесем словарь РуСентиЛекс в DataFrame.

In [2]:
with open('RuSentiLex2017_revised_2utf.txt', 'r') as f:
    dict_rsl = f.readlines()
dict_rsl = pd.DataFrame(
    [string.split(',') for string in dict_rsl[18:]],
    columns=['word', 'part_of_speech', 'lemma', 'sentiment', 'source', 'ambiguity', 'smth1', 'smth2']
)

In [3]:
def sentiment_calc(obj):
    obj = obj.replace(' ', '')
    if obj == 'positive':
        return 1
    elif obj == 'negative':
        return -1
    else:
        return 0

In [4]:
dict_rsl['sentiment'] = dict_rsl['sentiment'].apply(sentiment_calc)

In [5]:
dict_compar = dict_rsl[['lemma', 'sentiment']].copy()

In [6]:
vals = dict_compar['lemma'].values.tolist()
dict_compar_ = dict_compar[['sentiment']].copy()
dict_compar_.index = dict_compar['lemma']
dict_compar_map = dict_compar_.to_dict()['sentiment']

Собираем нужные файлы

In [7]:
# Функция для чтения набора файлов относящихся к n статье
def art_read(num):
    with open('Texts/art{}.txt'.format(num), 'r', encoding='utf-8') as f:
        text_lines = f.readlines()
        f.seek(0)
        text_full = f.read()

    with open('Texts/art{}.opin.txt'.format(num), 'r', encoding='utf-8') as f:
        opin_lines = f.readlines()
        f.seek(0)
        opin_full = f.read()

    with open('Texts/art{}.ann'.format(num), 'r') as f:
        entities_lines = f.readlines()
        f.seek(0)
        entities_full = f.read()
    return {'txt_l':text_lines, 'txt_f':text_full, 'opin_l':opin_lines,
            'opin_f':opin_full, 'ent_l':entities_lines, 'ent_f':entities_full}

In [8]:
art = art_read(1)
text_lines = art['txt_l']
text_full = art['txt_f']
ent_lines = art['ent_l']
ent_full = art['ent_f']

Рассмотрение сущностей

In [9]:
def entity_transform(lemmas):
    result = ''.join(lemmas)
    result = result.replace('\n', '')
    return result

m = Mystem()

def entity_pairs(ent_lines):
    entities = pd.DataFrame(
        [
            sum([re.findall(r'T[0-9]+\t([A-Z]+)?', string) for string in ent_lines], []),
            sum([re.findall(r'[0-9]+ [0-9]+\t(.+?)\n', string) for string in ent_lines], [])
        ], index = ['type', 'name']
    ).T
    entities = entities[(entities['name']!='Author') & (entities['name']!='Unknown')].dropna().reset_index(drop=True)
    entities['name'] = entities['name'].apply(m.lemmatize)
    entities['name'] = entities['name'].apply(entity_transform)
    entities_unique = entities.drop_duplicates()
    entities_unique_list = entities_unique['name'].values.tolist()
    list_of_pairs = [(entities_unique_list[p1], entities_unique_list[p2])
                     for p1 in range(len(entities_unique_list))
                     for p2 in range(p1+1,len(entities_unique_list))]
    return list_of_pairs

list_of_pairs = entity_pairs(ent_lines)

In [54]:
import pandas as pd
from pymystem3 import Mystem
df = pd.DataFrame()
m=Mystem()
for i in range(1,46):
    if(i in [9,22,26]): continue
    df2 = pd.read_json('Texts/art{}syn.json'.format(i))
    df2['file'] = i
    df = pd.concat([df,df2]) 


df = df.reset_index()
def get_struct_data(x):
    sentl =[]
    pars=[]
    for text in x:
        for sent in text:
            sentl = sent.split(' ')
            to_app = [sentl[i] for i in range(len(sentl)) if i in([0,1,4,6,8,10,12])]
            to_app.append(m.lemmatize(to_app[2])[0])
            pars.append(to_app)
    return(pars)
       
df['syntax'] = df['syntax'].apply(get_struct_data)  


def get_mutual_node(n_file,row,ent1,ent1pos,ent2,ent2pos):
    syntax = (df[(df['file']==n_file) & (df['index']==row)].syntax.iloc[0])
    
    def get_node(e,p):
        e = e.split(" ")[0]
        ent_dict = dict()
        for i in range(len(syntax)):
            if(syntax[i][7]==e):
                ent_dict[abs(int(syntax[i][1])-p)]=i
        return ent_dict[sorted(ent_dict)[0]]
    def get_route(i):
        route = list()
        for _ in range(len(syntax)):
            h=int(syntax[i][5])
            if(h ==-1): return route
            route.append(syntax[h])
            i=h
    print(ent1 + ent2)
    #print(syntax)
    ent_node1 = get_node(ent1,ent1pos)
    ent_node2 = get_node(ent2,ent2pos)
    route1= get_route(ent_node1)[::-1]
    route2= get_route(ent_node2)[::-1]
    if(len(route1)==0): return syntax[ent_node1],syntax[ent_node1],syntax[ent_node2]
    if(len(route2)==0): return syntax[ent_node2],syntax[ent_node1],syntax[ent_node2]

    for i in range(len(route1)):
        #print(i)
        if(route1[i][0]==route2[i][0]): obj=route1[i]
        else: break
    
    return obj,syntax[ent_node1],syntax[ent_node2]
    

Анализ первого текста

In [106]:
import re
def sentance_extraction(n_row,text, pairs):
    lemmas = m.lemmatize(text)
    sentence = ''.join(lemmas)
    sentence_ent = []
    for p in pairs:
        if (p[0] in sentence) & (p[1] in sentence):
            first_char = None
            
            #print(first_char)
            second_char = None
            regex1= re.compile(r"(?![с\\w\\d]){}(?![\\w\\d])".format(p[0]),flags=re.U)
            regex2= re.compile(r"(?![с\\w\\d]){}(?![\\w\\d])".format(p[1]),flags=re.U)
            for first_char in [m.start(0) for m in re.finditer(regex1, sentence)]:break
            for second_char in [m.start(0) for m in re.finditer(regex2, sentence)]:break
            print(first_char)
            print(second_char)
            
            if(not first_char or not second_char): continue
            get_mutual_node(1,n_row,p[0],first_char,p[1],second_char)
            sorted_ent = sorted(
                [
                    (first_char,p[0]), (second_char, p[1])
                ],
                key = lambda element : element[0]
            )

            if sorted_ent not in sentence_ent:
                sentence_ent.append(sorted_ent)

    result = []
    for p in sentence_ent:
        if p[1][0] - p[0][0] > 10:

            words = []
            for v in vals:
                if v in sentence[p[0][0]:p[1][0]]:
                    if v not in words:
                        words.append(v)

            result.append(
                [p[0][1],
                p[1][1],
                sum(list(map(lambda x: dict_compar_map[x], words))),
                text]
            )
    return result

In [108]:
text_lines = [line for line in text_lines if line != '\n']
opinions = []
for i in range(len(text_lines)):
    print("line"+str(i))
    print(text_lines[i])
    opinions += sentance_extraction(i-1,text_lines[i].replace("{Author, Unknown} ",""), list_of_pairs)
df[df['file']==1]

line0
{Author, Unknown} 

line1
{Author, Unknown} СМИ Ирана: Финляндия не хочет вступать в НАТО из страха перед Россией?

None
4
None
10
None
41
None
61
None
61
4
10
иранфинляндия
4
41
ираннато
4
61
иранроссия
4
61
иранроссия
10
41
финляндиянато
10
61
финляндияроссия
10
61
финляндияроссия
41
61
натороссия
41
61
натороссия
61
61
россияроссия
line2
{Author, Unknown} inosmi.ru

line3
{Author, Unknown} 5-го, 2016

line4
{Author, Unknown} Зачем американцам понадобилось перебросить 250 своих военных на северо-восток Сирии, к окрестностям города Румийлан, разбирался обозреватель Khorasan (

None
105
None
140
None
79
None
6
None
90
105
140
румийланKhorasan
105
79
румийланири


IndexError: list index out of range

In [107]:
text_lines = [line for line in text_lines if line != '\n']
opinions = []
for i in range(len(text_lines)):
    print("line"+str(i))
    print(text_lines[i])
    opinions += sentance_extraction(i-1,text_lines[i].replace("{Author, Unknown} ",""), list_of_pairs)

line0
{Author, Unknown} 

line1
{Author, Unknown} СМИ Ирана: Финляндия не хочет вступать в НАТО из страха перед Россией?

None
4
None
10
None
41
None
61
None
61
4
10
иранфинляндия
4
41
ираннато
4
61
иранроссия
4
61
иранроссия
10
41
финляндиянато
10
61
финляндияроссия
10
61
финляндияроссия
41
61
натороссия
41
61
натороссия
61
61
россияроссия
line2
{Author, Unknown} inosmi.ru

line3
{Author, Unknown} 5-го, 2016

line4
{Author, Unknown} Зачем американцам понадобилось перебросить 250 своих военных на северо-восток Сирии, к окрестностям города Румийлан, разбирался обозреватель Khorasan (

None
105
None
140
None
79
None
6
None
90
105
140
румийланKhorasan
105
79
румийланири


IndexError: list index out of range

In [40]:
df[df['file']==1]

,index,syntax,text,file
0,0,"[[0, 3, СМИ, NOUN, Animacy=Inan|Case=Nom|Gende...",СМИ Ирана: Финляндия не хочет вступать в НАТО ...,1
1,1,"[[0, 6, inosmi, ADJ, Animacy=Inan|Case=Nom|Gen...",inosmi.ru,1
2,10,"[[0, 5, Обама, NOUN, Animacy=Anim|Case=Nom|Gen...","Обама крайне нуждается в любых шагах, которые ...",1
3,11,"[[0, 1, C, ADP, Animacy=Inan|Case=Nom|Gender=F...","C другой стороны, сирийская армия при поддержк...",1
4,12,"[[0, 1, А, CONJ, fPOS=CONJ++, 7, cc, а], [2, 1...",А контроль над этими провинциями — это господс...,1
5,13,"[[0, 1, И, CONJ, fPOS=CONJ++, 3, cc, и], [2, 1...",И правительственная армия объявила о начале бо...,1
6,14,"[[0, 5, Таким, DET, Case=Ins|Gender=Masc|Numbe...","Таким образом, напрашивается вывод: если эти о...",1
7,15,"[[0, 2, По, ADP, fPOS=ADP++, 2, case, по], [3,...","По его мнению, на самом деле цель Белого дома ...",1
8,16,"[[0, 10, Изначально, ADV, Degree=Pos|fPOS=ADV+...",Изначально иранские СМИ сообщали об отказе Мос...,1
9,17,"[[0, 2, По, ADP, fPOS=ADP++, 1, case, по], [3,...","По соглашениям США и РФ, боевые действия в окр...",1


In [12]:
pd.DataFrame(opinions)

,0,1,2,3
0,сми,нато,-1,СМИ Ирана: Финляндия не хочет вступать в НАТО ...
1,сми,россия,-2,СМИ Ирана: Финляндия не хочет вступать в НАТО ...
2,иран,нато,-1,СМИ Ирана: Финляндия не хочет вступать в НАТО ...
3,иран,россия,-2,СМИ Ирана: Финляндия не хочет вступать в НАТО ...
4,финляндия,нато,-1,СМИ Ирана: Финляндия не хочет вступать в НАТО ...
5,финляндия,россия,-2,СМИ Ирана: Финляндия не хочет вступать в НАТО ...
6,нато,россия,-1,СМИ Ирана: Финляндия не хочет вступать в НАТО ...
7,сирия,румийлан,0,Зачем американцам понадобилось перебросить 250...
8,сирия,Khorasan,0,Зачем американцам понадобилось перебросить 250...
9,америка,сирия,0,Зачем американцам понадобилось перебросить 250...
